# RDF & SPARQL 

<a href="https://colab.research.google.com/github/joerg84/Graph_Powered_ML_Workshop/blob/master/Sparql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip3 install rdflib

In [2]:
import rdflib
from rdflib import Graph
from rdflib.namespace import DC, RDF, FOAF, RDFS
from rdflib import URIRef, BNode, Literal

# First Graph

In [ ]:
g = Graph()
# Graph using N3 syntax
n3data = """\
@prefix : <http://www.snee.com/ns/demo#> .
:Jane :hasParent :Gene .
:Gene :hasParent :Pat ;
      :gender    :female .
:Joan :hasParent :Pat ;
      :gender    :female .
:Pat  :gender    :male .
:Mike :hasParent :Joan ."""
g.parse(data=n3data, format="n3")
#print all triples
for s, p, o in g:
   print((s, p, o))

In [ ]:
# Lookup Jane by global identifier
jane = URIRef('http://www.snee.com/ns/demo#Jane')
print([o for o in g.predicate_objects(subject=jane)])

# Custom Graph

In [ ]:
bob = URIRef("http://example.org/people/Bob")
linda = BNode() #  Blank node with a autogenerated GUID

name = Literal('Bob') # passing a string
age = Literal(24) # passing a python int
height = Literal(76.5) # passing a python float

g = Graph()

g.add( (bob, RDF.type, FOAF.Person) )
g.add( (bob, FOAF.name, name) )
g.add( (bob, FOAF.knows, linda) )
g.add( (linda, RDF.type, FOAF.Person) )
g.add( (linda, FOAF.name, Literal('Linda') ) )

#print all triples
for s, p, o in g:
   print((s, p, o))

# Sparql

In [ ]:
result = g.query(
    """SELECT *
  WHERE
  {?s ?p ?o}
""")
for row in result:
    print(row)


In [ ]:
result = g.query(
    """SELECT DISTINCT ?aname ?bname
       WHERE {
          ?a foaf:knows ?b .
          ?a foaf:name ?aname .
          ?b foaf:name ?bname .
       }""", initNs={ 'foaf': FOAF })

for row in result:
    print("%s knows %s" % row)

# Import external Data

In [ ]:
g1 = rdflib.Graph()
g1.parse("http://www.w3.org/People/Berners-Lee/card")

print("Graph has %s statements." % len(g1))

for s, p, o in g1:
   print((s, p, o))

# RDF Schema

In [ ]:
g = Graph()
# Adapted from https://www.w3.org/TR/rdf-primer/
n3vehicledata = """\
@prefix ex: <http://example.org/schemas/vehicles#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
ex:MotorVehicle       rdf:type          rdfs:Class .
ex:PassengerVehicle   rdf:type          rdfs:Class .
ex:Van                rdf:type          rdfs:Class .
ex:Truck              rdf:type          rdfs:Class .
ex:MiniVan            rdf:type          rdfs:Class .

ex:PassengerVehicle   rdfs:subClassOf   ex:MotorVehicle .
ex:Van                rdfs:subClassOf   ex:MotorVehicle .
ex:Truck              rdfs:subClassOf   ex:MotorVehicle .

ex:MiniVan            rdfs:subClassOf   ex:Van .
ex:MiniVan            rdfs:subClassOf   ex:PassengerVehicle .
"""


g.parse(data=n3vehicledata, format="n3")
#print all triples
for s, p, o in g:
   print((s, p, o))

As rdfs:SubClassOf+ is transitive and reflexive, we can deduce facts which are not directly stated in the original triples.

In [ ]:


result = g.query(
    """SELECT DISTINCT ?s
  WHERE
  {
    ?s ?p ?o .
    ?s rdfs:subClassOf+ ex:MotorVehicle .
  }""", initNs={ 'rdfs': RDFS, 'rdf' : RDF, 'ex' : 'http://example.org/schemas/vehicles#' })


for row in result:
    print(row)



Note, that also MiniVan is showing up as MotorVehicle despite there is no direct fact about this. Feel free to check original statements about the subject subject MiniVan.